## Training and inference with YOLO_V3

In [0]:
#Let's mount the content from google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# This cell can be commented once you checked the current CUDA version
# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it. In some time from now maybe you 
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [0]:
# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
#Instaling cudNN. JUST ONCE!

#!tar -xzvf gdrive/My\ Drive/YOLO/cudNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/

#!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
#!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

In [0]:
# Downloading Darknet
!git clone https://github.com/kriyeng/darknet/
%cd darknet

# Check the folder
!dir

!git checkout feature/google-colab

#Compile Darknet
!make

Cloning into 'darknet'...
remote: Enumerating objects: 10068, done.
remote: Total 10068 (delta 0), reused 0 (delta 0), pack-reused 10068
Receiving objects: 100% (10068/10068), 10.14 MiB | 5.22 MiB/s, done.
Resolving deltas: 100% (6735/6735), done.
/content/darknet
3rdparty      CMakeLists.txt	      image_yolov3.sh	     scripts
appveyor.yml  CMakeSettings.json      include		     src
build	      DarknetConfig.cmake.in  json_mjpeg_streams.sh  video_v2.sh
build.ps1     darknet.py	      LICENSE		     video_yolov3.sh
build.sh      darknet_video.py	      Makefile
cfg	      data		      net_cam_v3.sh
cmake	      image_yolov2.sh	      README.md
Branch 'feature/google-colab' set up to track remote branch 'feature/google-colab' from 'origin'.
Switched to a new branch 'feature/google-colab'
mkdir -p obj
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -W

In [0]:
#Checking the device selected
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
#some functions to show, upload or download images to/from collab
def imShow(path):
    import cv2
    import matplotlib.pyplot as plt
    %matplotlib inline

    image = cv2.imread(path)
    height, width = image.shape[:2]
    resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.show()
    
def upload():
    from google.colab import files
    uploaded = files.upload() 
    for name, data in uploaded.items():
        with open(name, 'wb') as f:
        f.write(data)
        print('saved file', name)
        
def download(path):
    from google.colab import files
    files.download(path)

In [0]:
# Let's copy the train files needed to the local filesystem
!cp -r "/content/gdrive/My Drive/YOLO/train" ./train

In [0]:
# Let's copy the test files needed to the local filesystem
!cp -r "/content/gdrive/My Drive/YOLO/test" ./test

In [0]:
# Training the model from the previous weights used the last time
!./darknet detector train "/content/gdrive/My Drive/YOLO/obj.data" "/content/gdrive/My Drive/YOLO/yolov3_malaria.cfg" "/content/gdrive/My Drive/YOLO/backup/yolov3_malaria_last.weights" -map -dont_show 


 (next mAP calculation at 6600 iterations) 
 Last accuracy mAP@0.5 = 46.62 % 
 6597: 3.554048, 4.037333 avg loss, 0.001000 rate, 67.602885 seconds, 422208 images
Loaded: 0.000047 seconds

 (next mAP calculation at 6600 iterations) 
 Last accuracy mAP@0.5 = 46.62 % 
 6598: 4.336293, 4.067229 avg loss, 0.001000 rate, 67.955659 seconds, 422272 images
Loaded: 0.000047 seconds

 (next mAP calculation at 6600 iterations) 
 Last accuracy mAP@0.5 = 46.62 % 
 6599: 4.635157, 4.124022 avg loss, 0.001000 rate, 67.661389 seconds, 422336 images
Loaded: 0.000051 seconds

 (next mAP calculation at 6600 iterations) 
 Last accuracy mAP@0.5 = 46.62 % 
 6600: 4.450764, 4.156696 avg loss, 0.001000 rate, 67.447189 seconds, 422400 images
Resizing to initial size: 832 x 832 
 try to allocate additional workspace_size = 0.04 MB 
 CUDA allocate done! 
valid: Using default 'data/train.txt'
valid: Using default '/content/gdrive/My Drive/YOLO/train.txt'

 calculation mAP (mean average precision)...
1008
 detecti

### **TESTING TIME**

In [0]:
# Let's predict one image
!./darknet detector test "/content/gdrive/My Drive/YOLO/obj.data" "/content/gdrive/My Drive/YOLO/yolov3_malaria_test.cfg" "/content/gdrive/My Drive/YOLO/weights/yolov3_malaria_last.weights" test/Imagen61.png -thresh 0.2
imShow('predictions.jpg')

In [0]:
# Let's show the map metric
!./darknet detector map "/content/gdrive/My Drive/YOLO/obj.data" "/content/gdrive/My Drive/YOLO/yolov3_malaria_test.cfg" "/content/gdrive/My Drive/YOLO/backup/yolov3_malaria_last.weights"